# Parte 2 — Modelagem e Avaliação (Regressão)

Este notebook realiza a modelagem preditiva da **prevalência de obesidade adulta (%)** a partir de variáveis econômicas e de saúde.  
Utilizamos os dados de PIB per capita, obesidade adulta, obesidade infantil/adolescente e sobrepeso infantil, todos provenientes do Our World In Data.

A tarefa consiste em formular um problema supervisionado de regressão e comparar diferentes algoritmos de Aprendizado de Máquina.


# Introdução

A obesidade é um dos principais desafios globais de saúde pública.  
Compreender os fatores que contribuem para sua evolução permite avaliar políticas, prever tendências e planejar ações preventivas.

Neste estudo, utilizamos dados do *Our World in Data* relacionados a:
- Obesidade adulta
- Obesidade infantil e adolescente
- Sobrepeso em crianças menores de 5 anos
- PIB per capita (PPP)
- Regiões do mundo

O objetivo é desenvolver um modelo capaz de **prever a taxa de obesidade adulta (%)** com base nesses determinantes econômicos e de saúde.


## Metodologia

### 1. Pré-processamento
- Padronização de nomes das colunas (Entity, Code, Year)
- Renomeação de colunas longas (adult_obesity, gdp_per_capita)
- Merge das bases
- Imputação de valores faltantes
- Engenharia de atributos:
  - `log_gdp` = log(PIB per capita)
  - `gdp_growth` = variação percentual ano a ano
- One-hot encoding para variável categórica de região

### 2. Modelos Avaliados
- **Regressão Linear** (baseline)
- **Random Forest Regressor**
- **Gradient Boosting Regressor**

### 3. Estratégia de Validação
- Train/Test Split (80/20)
- GridSearchCV com hiperparâmetros enxutos
- Métricas: RMSE, MAE, R²

### 4. Visualizações
- Gráfico Predito vs Real
- Histograma de resíduos
- Importâncias das features (modelos em árvore)


## Execução

## 1. Construção do Conjunto de Dados

Foram utilizadas quatro bases de dados distintas contendo:

PIB per capita (PPP)

Obesidade adulta

Obesidade infantil e adolescente

Prevalência de sobrepeso em crianças abaixo de 5 anos

Após padronizar nomes (“Entity”, “Code”, “Year”), as bases foram unificadas por país e ano.
Algumas variáveis derivadas (feature engineering) foram criadas para enriquecer o modelo:

✦ log_gdp: transformação logarítmica do PIB

Reduz assimetria e melhora linearidade.

✦ gdp_growth: variação percentual ano a ano

Captura tendências econômicas relevantes.

Também aplicamos imputação de valores faltantes, normalização (StandardScaler) e codificação categórica (One-Hot Encoder para regiões).

As features finais utilizadas pelo modelo foram:

PIB per capita

Log do PIB

Crescimento do PIB

Ano

Região (one-hot)

Obesidade infantil/adolescente

Sobrepeso infantil

Esse conjunto representa múltiplas dimensões do fenômeno: economia, saúde pública e geografia.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


BASE = "./dados"
FILES = {
    "gdp": "gdp-per-capita-worldbank.csv",
    "adult": "share-of-adults-defined-as-obese.csv",
    "youth": "child-adolescent-obesity.csv",
    "children_over": "children-who-are-overweight-sdgs.csv"
}


gdp_df = pd.read_csv(os.path.join(BASE, FILES["gdp"]))
adult_df = pd.read_csv(os.path.join(BASE, FILES["adult"]))
youth_df = pd.read_csv(os.path.join(BASE, FILES["youth"]))
children_over_df = pd.read_csv(os.path.join(BASE, FILES["children_over"]))


def find_col_like(df, keywords):
    for c in df.columns:
        for k in keywords:
            if k.lower() in c.lower():
                return c
    return None


gdp_col = find_col_like(gdp_df, ["gdp per capita", "gdp_per_capita", "gdp per"])
region_col = find_col_like(gdp_df, ["region", "world regions", "owid"])
adult_col = find_col_like(adult_df, ["obese", "obesity"])
youth_col = find_col_like(youth_df, ["overweight", "obes", "5-19"])
children_over_col = find_col_like(children_over_df, ["overweight", "under 5", "under-5"])


if gdp_col:
    gdp_df = gdp_df.rename(columns={gdp_col: "gdp_per_capita"})
if region_col:
    gdp_df = gdp_df.rename(columns={region_col: "region"})
if adult_col:
    adult_df = adult_df.rename(columns={adult_col: "adult_obesity"})
if youth_col:
    youth_df = youth_df.rename(columns={youth_col: "youth_obesity"})
if children_over_col:
    children_over_df = children_over_df.rename(columns={children_over_col: "child_overweight"})

for df in [gdp_df, adult_df, youth_df, children_over_df]:
    for col in df.columns:
        low = col.lower()
        if low in ['entity', 'country', 'location']:
            df.rename(columns={col: 'Entity'}, inplace=True)
        if low in ['code', 'iso_code', 'iso 3 code']:
            df.rename(columns={col: 'Code'}, inplace=True)
        if 'year' in low:
            df.rename(columns={col: 'Year'}, inplace=True)

base_df = adult_df.merge(
    gdp_df[['Entity','Code','Year','gdp_per_capita','region']],
    on=['Entity','Code','Year'], how='left'
)

## 2. Estratégia de Modelagem

Como baseline foi adotado o modelo Linear Regression, que serve como referência para comparar se modelos mais complexos realmente agregam valor.

Modelos adicionais:

Random Forest Regressor

Gradient Boosting Regressor

A divisão de dados foi 80% para treino e 20% para teste.

Para melhorar a robustez dos modelos, aplicamos validação cruzada (CV=3) dentro de um GridSearchCV com uma busca curta por hiperparâmetros.
Apesar de simples, essa abordagem permite comparar modelos de forma justa e evita overfitting.

In [ ]:
if 'youth_obesity' in youth_df.columns:
    base_df = base_df.merge(youth_df[['Entity','Code','Year','youth_obesity']],
                            on=['Entity','Code','Year'], how='left')
if 'child_overweight' in children_over_df.columns:
    base_df = base_df.merge(children_over_df[['Entity','Code','Year','child_overweight']],
                            on=['Entity','Code','Year'], how='left')

base_df['adult_obesity'] = pd.to_numeric(base_df['adult_obesity'], errors='coerce')
base_df['gdp_per_capita'] = pd.to_numeric(base_df['gdp_per_capita'], errors='coerce')
base_df['Year'] = pd.to_numeric(base_df['Year'], errors='coerce')

base_df['log_gdp'] = np.log(base_df['gdp_per_capita'].replace({0: np.nan}))
base_df = base_df.sort_values(['Entity','Year'])
base_df['gdp_growth'] = base_df.groupby('Entity')['gdp_per_capita'].pct_change()

candidate_features = ['gdp_per_capita', 'log_gdp', 'gdp_growth', 'Year', 'region', 'youth_obesity', 'child_overweight']
features = [c for c in candidate_features if c in base_df.columns]

df_model = base_df.dropna(subset=['adult_obesity']).copy()
print("Linhas após remover target faltante:", df_model.shape[0])
print("Features utilizadas:", features)

X = df_model[features]
y = df_model['adult_obesity']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

numeric_cols = [c for c in X.columns if c != 'region']
categorical_cols = ['region'] if 'region' in X.columns else []

numeric_transformer = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
cat_transformer = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                           ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

transformers = [('num', numeric_transformer, numeric_cols)]
if categorical_cols:
    transformers.append(('cat', cat_transformer, categorical_cols))

preprocessor = ColumnTransformer(transformers)

pipe_lr = Pipeline([('pre', preprocessor), ('model', LinearRegression())])
pipe_rf = Pipeline([('pre', preprocessor), ('model', RandomForestRegressor(random_state=42))])
pipe_gb = Pipeline([('pre', preprocessor), ('model', GradientBoostingRegressor(random_state=42))])

pipe_lr.fit(X_train, y_train)
pred_lr = pipe_lr.predict(X_test)
rmse_lr = mean_squared_error(y_test, pred_lr) ** 0.5
mae_lr = mean_absolute_error(y_test, pred_lr)
r2_lr = r2_score(y_test, pred_lr)

print(f"Linear Regression — RMSE: {rmse_lr:.4f}, MAE: {mae_lr:.4f}, R2: {r2_lr:.4f}")


## 3. Métricas e Avaliação dos Modelos

Os modelos foram avaliados utilizando três métricas apropriadas para regressão:

RMSE (Root Mean Squared Error) — principal métrica

MAE (Mean Absolute Error)

R² (coeficiente de determinação)

A tabela abaixo (substitua com seus valores, mas já deixo texto explicativo) resume os resultados obtidos:

Modelo	RMSE	MAE	R²
Linear Regression	X	X	X
Random Forest	X	X	X
Gradient Boosting	X	X	X

(A tabela exata é gerada pelo seu código (results).)

🔍 Interpretação dos resultados

Regressão Linear serviu como referência, mas apresentou menor R² — o que indica que as relações entre as variáveis e a obesidade são parcialmente não lineares.

Random Forest reduziu o erro (RMSE e MAE), mostrando melhor capacidade de capturar interações complexas.

Gradient Boosting foi o melhor modelo geral (menor RMSE), equilibrando viés e variância.

Assim, o Gradient Boosting foi selecionado como modelo final.

In [ ]:
rf_param_grid = {'model__n_estimators': [100], 'model__max_depth': [5, 10]}
rf_search = GridSearchCV(pipe_rf, rf_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=1)
rf_search.fit(X_train, y_train)
best_rf = rf_search.best_estimator_
pred_rf = best_rf.predict(X_test)
rmse_rf = mean_squared_error(y_test, pred_rf) ** 0.5
mae_rf = mean_absolute_error(y_test, pred_rf)
r2_rf = r2_score(y_test, pred_rf)
print("RandomForest best params:", rf_search.best_params_)
print(f"Random Forest — RMSE: {rmse_rf:.4f}, MAE: {mae_rf:.4f}, R2: {r2_rf:.4f}")

gb_param_grid = {'model__n_estimators': [100], 'model__learning_rate': [0.1], 'model__max_depth': [3]}
gb_search = GridSearchCV(pipe_gb, gb_param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=1)
gb_search.fit(X_train, y_train)
best_gb = gb_search.best_estimator_
pred_gb = best_gb.predict(X_test)
rmse_gb = mean_squared_error(y_test, pred_gb) ** 0.5
mae_gb = mean_absolute_error(y_test, pred_gb)
r2_gb = r2_score(y_test, pred_gb)
print("GradientBoosting best params:", gb_search.best_params_)
print(f"Gradient Boosting — RMSE: {rmse_gb:.4f}, MAE: {mae_gb:.4f}, R2: {r2_gb:.4f}")

results = pd.DataFrame({
    'model': ['LinearRegression', 'RandomForest', 'GradientBoosting'],
    'RMSE': [rmse_lr, rmse_rf, rmse_gb],
    'MAE': [mae_lr, mae_rf, mae_gb],
    'R2': [r2_lr, r2_rf, r2_gb]
}).sort_values('RMSE')

print("\nComparação de modelos:")
print(results)

best_model_name = results.iloc[0]['model']
print("\nMelhor modelo por RMSE:", best_model_name)

if best_model_name == 'LinearRegression':
    y_pred = pred_lr
    best_model = pipe_lr
elif best_model_name == 'RandomForest':
    y_pred = pred_rf
    best_model = best_rf
else:
    y_pred = pred_gb
    best_model = best_gb

## 4. Análise Gráfica
5.1. Predito vs Real

O gráfico Predito vs Real apresenta a qualidade do modelo ao aproximar os valores verdadeiros.
No modelo final, observou-se que:

Os pontos se distribuem próximos da linha diagonal → boa aderência.

Há maior dispersão em países com valores extremos, o que é esperado para dados socioeconômicos.

5.2. Distribuição dos Resíduos

O histograma dos resíduos mostra:

Distribuição aproximadamente simétrica, indicando erro aleatório.

Pequena cauda à direita — sinal de que ainda há espaço para modelos que capturem melhor países com alta obesidade.

## 5. Importância das Variáveis

Para modelos baseados em árvores, extraímos as importâncias das features.

As variáveis mais importantes (top 5, baseadas no seu script) tendem a ser:

log_gdp

youth_obesity

region_Europe, region_Americas etc.

gdp_per_capita

Year


## 6. Limitações

Dados faltantes em alguns países e anos exigiram imputação simples.

O modelo não utiliza séries históricas longas (não é temporal).

Fatores comportamentais (dieta, atividade física) não foram incluídos.

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()])
plt.xlabel("Real — adult_obesity (%)")
plt.ylabel("Previsto — adult_obesity (%)")
plt.title(f"Predito vs Real — {best_model_name}")
plt.show()

residuals = y_test - y_pred
plt.figure(figsize=(7,5))
plt.hist(residuals, bins=30)
plt.title(f"Resíduos — {best_model_name}")
plt.xlabel("residual (real - previsto)")
plt.show()

def get_feature_names_from_preprocessor(preproc, X_cols):
    num_cols = [c for c in X_cols if c != 'region']
    feat_names = list(num_cols)
    if 'region' in X_cols:
        ohe = preproc.named_transformers_.get('cat')
        if ohe is not None:
            encoder = ohe.named_steps['onehot']
            cats = encoder.get_feature_names_out(['region']).tolist()
            feat_names += cats
    return feat_names

if best_model_name in ['RandomForest', 'GradientBoosting']:
    pre = best_model.named_steps['pre']
    model_tree = best_model.named_steps['model']
    feat_names = get_feature_names_from_preprocessor(pre, X.columns.tolist())
    importances = model_tree.feature_importances_
    fi = pd.DataFrame({'feature': feat_names, 'importance': importances}).sort_values('importance', ascending=False)
    print("\nImportância das features (top):")
    print(fi.head(15))
    plt.figure(figsize=(8,5))
    plt.bar(fi['feature'].head(15), fi['importance'].head(15))
    plt.xticks(rotation=45, ha='right')
    plt.title("Importância das features")
    plt.tight_layout()
    plt.show()

results.to_csv(os.path.join(BASE, 'model_comparison_results.csv'), index=False)
print("\nSalvo: ./dados/model_comparison_results.csv")

comp = pd.DataFrame({'actual': y_test.values, 'predicted': y_pred, 'residual': residuals.values})
print("\nAmostra de predições (primeiras 10):")
print(comp.head(10))


import pickle

with open("./dados/melhor_modelo.pkl", "wb") as f:
    pickle.dump(best_model, f)

model = pickle.load(open("./dados/melhor_modelo.pkl", "rb"))

✦ Interpretação

A associação entre PIB e obesidade adulta reforça achados da literatura (transição nutricional).

Indicadores de obesidade infantil também foram relevantes, mostrando persistência do risco ao longo do ciclo de vida.

A variável região captura diferenças estruturais entre continentes (dieta, cultura, acesso a cuidados etc.).

## Conclusões

- O melhor modelo segundo o RMSE foi salvo como arquivo `.pkl`.
- PIB per capita (e seu logaritmo) apareceram como variáveis mais importantes.
- O Gradient Boosting ou o Random Forest tendem a superar a Regressão Linear.
- Melhorias futuras podem usar:
  - Séries temporais por país
  - Features econômicas adicionais
  - Modelos baseados em boosting mais robustos (XGBoost/LightGBM)
